SETTING UP THE ENVIRONMENT

In [ ]:
!pip install langchain
!pip install cohere tiktoken openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.3 MB/s eta 0:00:00


In [ ]:
!pip install python-dotenv

In [ ]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
import pandas as pd


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
#MOUNTING OUR DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv", encoding='latin-1')

In [ ]:
df.shape


(12625, 13)

In [ ]:
df.head()

,comment,comment_id,like_count,reply_count,published_at,parent_id,video_id,title,channel,publish_date,views,likes,comments_count
0,"So , do we have a forest in Ghana by name WONTUMI FOREST ? WOW, GHANAIANS ARE NOT SERIOUS AT ALL COST",UgwwfkJtgmu2XWedKG54AaABAg,0.0,0.0,2024-10-17T19:17:10Z,NaN,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0
1,Backwards society of leaders... polluting your own water bodies and calling for God's intervention. This is a man made disaster and man must take it's own action. It has nothing to do with God.????,UgwU_kn1hnQ020jABdx4AaABAg,2.0,1.0,2024-10-15T19:08:41Z,NaN,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0
2,Exactly ??,UgwU_kn1hnQ020jABdx4AaABAg.A9cUk3_TA8xA9jXMYU8VHm,0.0,0.0,2024-10-18T12:46:14Z,UgwU_kn1hnQ020jABdx4AaABAg,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0
3,kpaako???,UgzDOCWqsoEA5kaCZpN4AaABAg,0.0,0.0,2024-10-13T04:05:41Z,NaN,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0
4,YOU CAN'T DO THE RIVERS AND LEAVE THE FOREST UNTIL ANOTHER TIME. IT WILL BE BETTER IF YOU DON'T START AT ALL. THE PRESSURE THAT WILL COME ON THE FOREST EVEN IF ONE WEEK IS ALLOWED WILL BE VERY DISASTROUS. UNLESS YOU WANT GHANAIANS TO KNOW THAT SOME ARE BEING PROTECTED.,Ugw-i4E2ucWcPSsKV6J4AaABAg,1.0,0.0,2024-10-12T16:35:54Z,NaN,m3Q-UZQF56M,Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24),JoyNews,10/11/2024 19:35,10422.0,65.0,28.0


LANG CHAIN

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', encoding='latin-1')

In [ ]:
docs=loader.load()

In [ ]:
len(docs)

12625

In [ ]:
print(docs[0].page_content[:1000])

comment: So , do we have a forest in Ghana  by name WONTUMI FOREST ?  WOW,  GHANAIANS ARE NOT SERIOUS  AT ALL  COST
comment_id: UgwwfkJtgmu2XWedKG54AaABAg
like_count: 0
reply_count: 0
published_at: 2024-10-17T19:17:10Z
parent_id: 
video_id: m3Q-UZQF56M
title: Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24)
channel: JoyNews
publish_date: 10/11/2024 19:35
views: 10422
likes: 65
comments_count: 28


SPLITTING DOCS INTO CHUCKS BEFORE EMBEDDING

In [ ]:
chunk_size = 256
chunk_overlap = 32

In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [ ]:
c_text_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator=' ',
    length_function=len
    )

In [ ]:
r_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    add_start_index=True

)


In [ ]:
pages = c_text_splitter.split_documents(docs)

In [ ]:
print(docs[0])

page_content='comment: So , do we have a forest in Ghana  by name WONTUMI FOREST ?  WOW,  GHANAIANS ARE NOT SERIOUS  AT ALL  COST
comment_id: UgwwfkJtgmu2XWedKG54AaABAg
like_count: 0
reply_count: 0
published_at: 2024-10-17T19:17:10Z
parent_id: 
video_id: m3Q-UZQF56M
title: Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24)
channel: JoyNews
publish_date: 10/11/2024 19:35
views: 10422
likes: 65
comments_count: 28' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0}


In [ ]:
print(pages[0])

page_content='comment: So , do we have a forest in Ghana by name WONTUMI FOREST ? WOW, GHANAIANS ARE NOT SERIOUS AT ALL COST
comment_id: UgwwfkJtgmu2XWedKG54AaABAg
like_count: 0
reply_count: 0
published_at: 2024-10-17T19:17:10Z
parent_id: 
video_id: m3Q-UZQF56M
title:' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0}


In [ ]:
print(pages[1])

page_content='video_id: m3Q-UZQF56M
title: Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24)
channel: JoyNews
publish_date: 10/11/2024 19:35
views: 10422
likes: 65
comments_count: 28' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0}


In [ ]:
pages = r_text_splitter.split_documents(docs)

In [ ]:
print(pages[0])

page_content='comment: So , do we have a forest in Ghana  by name WONTUMI FOREST ?  WOW,  GHANAIANS ARE NOT SERIOUS  AT ALL  COST
comment_id: UgwwfkJtgmu2XWedKG54AaABAg
like_count: 0
reply_count: 0
published_at: 2024-10-17T19:17:10Z
parent_id: 
video_id: m3Q-UZQF56M' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0, 'start_index': 0}


In [ ]:
print(pages[1])

page_content='video_id: m3Q-UZQF56M
title: Galamsey Fight: Joint task force declares relentless war on illegal miners | The Pulse (11-10-24)
channel: JoyNews
publish_date: 10/11/2024 19:35
views: 10422
likes: 65
comments_count: 28' metadata={'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'row': 0, 'start_index': 231}


In [ ]:
print(len(docs))
print(len(pages))

12625
18569


TURNING WORDS INTO NUMBERS - EMBEDDINGS

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:0

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
embeddings = OpenAIEmbeddings()

<ipython-input-31-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


FOLDER TO STORE EMBEDDINGS

In [ ]:
persist_directory = 'persist_chroma'

CONVERT EMBEDDINGS INTO DB and store in directory for future use

In [ ]:
vectordb = Chroma.from_documents(
    documents=pages,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

18569


In [ ]:
vectordb.persist()

<ipython-input-35-d832e715b29a>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


Querying our vector database

In [ ]:
question="which comments talk about water bodies?"

In [ ]:
docs = vectordb.similarity_search(question, k=3) #k top number to return

In [ ]:
docs

[Document(metadata={'row': 12622, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: Ghanaians stop distory water bodys. No water No Life\ncomment_id: UgzcOFfjRKtJrgotTUl4AaABAg\nlike_count: 3\nreply_count: 0\npublished_at: 2021-06-03T19:04:42Z\nparent_id: \nvideo_id: yUPB3-s6Tb4'),
 Document(metadata={'row': 12601, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 224}, page_content="country, but they didn't tell anybody. If they were doing it very well without destroying our lands, & our water bodies, who will care??"),
 Document(metadata={'row': 12612, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: Let them be vigilant, not to allow any galamsey to go on in the country.\ncomment_id: Ugw0qfavz4NEaBUi3op4AaABAg.9O8TKctAZSs9O8Z6bD7hn3\nlike_count: 0\nreply_count: 0\npublished_at: 2021-06-03T21:12:04Z\nparent_id: Ugw0qfavz4NEaBUi3o

SIMILARITY CHECK

In [ ]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings

)

<ipython-input-39-c2a71514b216>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [ ]:
question = "Comments that talk about governance?"

In [ ]:
doc_similarity = vectordb.similarity_search_with_score(question,k=3)
doc_similarity

[(Document(metadata={'row': 12357, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content="comment: Government controlled everything and shared among themselves the leaders don't think about the country they worry about their fat bank account . Shame !\ncomment_id: Ugw9bCMhfDor152zth54AaABAg\nlike_count: 0\nreply_count: 0"),
  0.335551381111145),
 (Document(metadata={'row': 10437, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: This is responsible government, three gbosa? gbosa? gbosa? for Ghanaians. Daily your Government shows us (Nigerians) how ours had failed?? the citizens.\ncomment_id: UgzZwNxM4PrkShiWPpx4AaABAg\nlike_count: 0\nreply_count: 0'),
  0.3445374667644501),
 (Document(metadata={'row': 11976, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content="comment: The people we call leaders don't care but their pockets very sa

MAX MARGINAL RELEVANCE - MMR

In [ ]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
docs_mmr

[Document(metadata={'row': 12357, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content="comment: Government controlled everything and shared among themselves the leaders don't think about the country they worry about their fat bank account . Shame !\ncomment_id: Ugw9bCMhfDor152zth54AaABAg\nlike_count: 0\nreply_count: 0"),
 Document(metadata={'row': 9804, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: Poor governance..... You wait for the rivers are destroyed b4.  Awww Ghana ????\ncomment_id: Ugye2sSNUb0dpmDOgYJ4AaABAg\nlike_count: 24\nreply_count: 13\npublished_at: 2021-05-01T14:44:08Z\nparent_id: \nvideo_id: GUqQ_x13nJE'),
 Document(metadata={'row': 9958, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content="comment: Actually, the 'head' chief and the people from (and living in) affected communities can do better at addressing 

RAG FOR QUESTION ANSWERING

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

import langchain
langchain.verbose =True #Enable debugging

SPECIFY model details

In [ ]:
#llm_name = "gpt-3.5-turbo"
llm_name = "gpt-4"

llm = ChatOpenAI(model_name=llm_name, temperature=1)

<ipython-input-44-b48b9f20b3a4>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_name, temperature=1)


CREATE A QA CHAIN WITH RETRIEVAL QA

In [ ]:
qa_chain_default = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={'k':3}),
    chain_type="stuff",
    return_source_documents=True
)

In [ ]:
question = "Which comments talk about water bodies?and what are their titles?"

In [ ]:
result = qa_chain_default({'query':question})

<ipython-input-47-9c5f74536312>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain_default({'query':question})




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
comment: Ghanaians stop distory water bodys. No water No Life
comment_id: UgzcOFfjRKtJrgotTUl4AaABAg
like_count: 3
reply_count: 0
published_at: 2021-06-03T19:04:42Z
parent_id: 
video_id: yUPB3-s6Tb4

country, but they didn't tell anybody. If they were doing it very well without destroying our lands, & our water bodies, who will care??

comment: How?
comment_id: UgxdaNyvZzzS7HCPSHt4AaABAg.9O8VZlMwU7T9OKNF4EUzwz
like_count: 0
reply_count: 0
published_at: 2021-06-08T11:19:15Z
parent_id: UgxdaNyvZzzS7HCPSHt4AaABAg
video_id: yUPB3-s6Tb4
Human: Which comments talk about water bodies?and what are their titles?

> Finished chain.

> Finished chain.

> Finished chain.


MAKING THE RESULT PRETTIER

Pretty Print Function

In [ ]:
def pretty_print(text, words_per_line=15):
  #splitting the input intp words
  words = text.split()
  #lines = []
  #current_line = words[0]
  for i in range(0, len(words), words_per_line):
    line = ' '.join(words[i:i+words_per_line])
    print(line)

In [ ]:
pretty_print(result['result'])

The comments that discuss water bodies are: 1) Comment title: "Ghanaians stop distory water bodys.
No water No Life" Comment ID: UgzcOFfjRKtJrgotTUl4AaABAg 2) Comment title: "country, but they didn't tell
anybody. If they were doing it very well without destroying our lands, & our water
bodies, who will care??" Comment ID: There is no comment ID provided for this comment.


CUSTOM TEMPLATE - CONVERSATION REQUIRES USER, SYSTEM AND AI ASSISTANT


In [ ]:
 from langchain.schema import SystemMessage
 from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

SETTING THE STAGE FOR THE AI BEHAVIOUR

In [ ]:
system_msg = """
You are a helpful assistant Kay using the context from a private dataset to answer questions. If you do not know
the answer, reply "No direct answer found."

Start your answer with a brief greeting, for example, "Hi I am chatbot Kay. Thanks for asking."

Always end your answer with "Let me Know if you have further questions."

Be Concise."""

HUMAN MESSAGE CONTEXT{context} and {question} are place holders

In [ ]:
 human_template = """
 Use the folowing pieces of context to answer the question at the end.
 ###
 {context}

 Question: {question}
 Answer:"""

Chat Prompt Template passed to the LLM

In [ ]:
messages = ChatPromptTemplate.from_messages([
    SystemMessage(content=system_msg),
    HumanMessagePromptTemplate.from_template(human_template)
])

Custom Chain

In [ ]:
qa_chain_custom = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={'k':3}),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={'prompt':messages}
)

Testing our prompts

In [ ]:
question = "Which comments talk about water bodies?and what are their titles?"

In [ ]:
result = qa_chain_custom({'query':question})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant Kay using the context from a private dataset to answer questions. If you do not know 
the answer, reply "No direct answer found."

Start your answer with a brief greeting, for example, "Hi I am chatbot Kay. Thanks for asking."

Always end your answer with "Let me Know if you have further questions."

Be Concise.
Human: 
Use the folowing pieces of context to answer the question at the end.
###
comment: Ghanaians stop distory water bodys. No water No Life
comment_id: UgzcOFfjRKtJrgotTUl4AaABAg
like_count: 3
reply_count: 0
published_at: 2021-06-03T19:04:42Z
parent_id: 
video_id: yUPB3-s6Tb4

country, but they didn't tell anybody. If they were doing it very well without destroying our lands, & our water bodies, who will care??

comment: How?
comment_id: UgxdaNyvZzzS7HCPSHt4AaABAg.9O8VZlMwU7T9OKNF4EUzwz
like_coun

In [ ]:
result

{'query': 'Which comments talk about water bodies?and what are their titles?',
 'result': 'Hi, I am chatbot Kay. Thanks for asking.\n\nThe comments that talk about water bodies are:\n\n1. "Ghanaians stop distory water bodys. No water No Life" with the comment id: UgzcOFfjRKtJrgotTUl4AaABAg\n\n2. And the one that says "country, but they didn\'t tell anybody. If they were doing it very well without destroying our lands, & our water bodies, who will care??" This comment does not have a provided id in the dataset.\n\nLet me know if you have further questions.',
 'source_documents': [Document(metadata={'row': 12622, 'source': '/content/drive/MyDrive/Thesis_Dataset/genai_sampler.csv', 'start_index': 0}, page_content='comment: Ghanaians stop distory water bodys. No water No Life\ncomment_id: UgzcOFfjRKtJrgotTUl4AaABAg\nlike_count: 3\nreply_count: 0\npublished_at: 2021-06-03T19:04:42Z\nparent_id: \nvideo_id: yUPB3-s6Tb4'),
  Document(metadata={'row': 12601, 'source': '/content/drive/MyDrive/Th

In [ ]:
pretty_print(result['result'])

Hi, I am chatbot Kay. Thanks for asking. The comments that talk about water bodies
are: 1. "Ghanaians stop distory water bodys. No water No Life" with the comment id:
UgzcOFfjRKtJrgotTUl4AaABAg 2. And the one that says "country, but they didn't tell anybody. If they
were doing it very well without destroying our lands, & our water bodies, who will
care??" This comment does not have a provided id in the dataset. Let me know
if you have further questions.
